In [3]:
import pandas as pd

In [2]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls"

File '2014_All_CAP_Search_Results_Data_P14.xls' already there; not retrieving.



In [1]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls"

--2016-06-02 12:01:45--  http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls
Resolving cap-payments.defra.gov.uk... 194.1.210.50
Connecting to cap-payments.defra.gov.uk|194.1.210.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33282560 (32M) [application/vnd.ms-excel]
Saving to: '2015_All_CAP_Search_Results_Data_P14.xls'

2015_All_CAP_Search 100%[===================>]  31.74M  1.91MB/s    in 12s     

2016-06-02 12:01:57 (2.72 MB/s) - '2015_All_CAP_Search_Results_Data_P14.xls' saved [33282560/33282560]



In [26]:
YEAR = 2015

In [19]:
excel_file = pd.ExcelFile('%s_All_CAP_Search_Results_Data_P14.xls' % YEAR)

In [20]:
id_vars = ['Year', 'BeneficiaryCode', 'BeneficiaryName_F201',
       'PostcodePrefix_F202B', 'TownCity_F202C',
#        'OtherEAGFTotal', 'DirectEAGFTotal', 'RuralDevelopmentTotal', 'Total',
        'PayingAgencyLink']


def get_melted_df(excel_file, sheet_name):
    df_raw = excel_file.parse(sheet_name)
    df_raw = df_raw[df_raw['Year'].notnull()]
    all_columns = list(df_raw.columns)
    value_vars = list(set(all_columns) - set(id_vars))
    df_raw = df_raw.rename(columns={x: x.strip() for x in value_vars})
    value_vars = [v.strip() for v in value_vars]
    df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, var_name='scheme', value_name='amount')
    df = df[df['amount'].notnull()]
    return df

df = pd.concat([get_melted_df(excel_file, sheet) for sheet in excel_file.sheet_names])
df.head()

,Year,BeneficiaryCode,BeneficiaryName_F201,PostcodePrefix_F202B,TownCity_F202C,PayingAgencyLink,scheme,amount
9552,2014.0,NaN,CALEDON ESTATES CO,BT68,CALEDON,DARDNI,Non-productive investments,2900.0
9567,2014.0,NaN,CARRICKFERGUS BOROUGH COUNCIL,BT38,CARRICKFERGUS,DARDNI,Non-productive investments,1580.0
9755,2014.0,NaN,DR S PETER FITZGERALD,BT29,CRUMLIN,DARDNI,Non-productive investments,3000.0
9781,2014.0,NaN,DUNLEATH ESTATES LTD,BT22,NEWTOWNARDS,DARDNI,Non-productive investments,912.5
114028,2014.0,1219336.0,*******,BT62,CRAIGAVON,DARDNI,First afforestation of agricultural land,267.8


In [21]:
df = df.rename(columns={
    'Year': 'year',
    'BeneficiaryCode': 'recipient_id',
    'BeneficiaryName_F201': 'recipient_name',
    'PostcodePrefix_F202B': 'recipient_postcode',
    'TownCity_F202C': 'recipient_location',
    'PayingAgencyLink': 'agency',
})
df['country'] = 'GB'
df['currency'] = 'GBP'
df['year'] = df['year'].astype('int')
df['recipient_name'] = df['recipient_name'].str.strip()
df['recipient_postcode'] = df['recipient_postcode'].str.strip()
df['recipient_location'] = df['recipient_location'].str.strip()
df.set_value(df['recipient_id'].notnull(), 'recipient_name', None)
df.head()

,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
9552,2014,NaN,CALEDON ESTATES CO,BT68,CALEDON,DARDNI,Non-productive investments,2900.0,GB,GBP
9567,2014,NaN,CARRICKFERGUS BOROUGH COUNCIL,BT38,CARRICKFERGUS,DARDNI,Non-productive investments,1580.0,GB,GBP
9755,2014,NaN,DR S PETER FITZGERALD,BT29,CRUMLIN,DARDNI,Non-productive investments,3000.0,GB,GBP
9781,2014,NaN,DUNLEATH ESTATES LTD,BT22,NEWTOWNARDS,DARDNI,Non-productive investments,912.5,GB,GBP
114028,2014,1219336.0,None,BT62,CRAIGAVON,DARDNI,First afforestation of agricultural land,267.8,GB,GBP


In [22]:
def set_recipient_id(row):
    if pd.isnull(row['recipient_id']):
        row['recipient_id'] = 'GB-%s-%s' % (row['recipient_postcode'], row['recipient_name'])
    else:
        row['recipient_id'] = 'GB-%s' % int(row['recipient_id'])
    return row

df = df.apply(set_recipient_id, axis=1)
df.head()

,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
9552,2014,GB-BT68-CALEDON ESTATES CO,CALEDON ESTATES CO,BT68,CALEDON,DARDNI,Non-productive investments,2900.0,GB,GBP
9567,2014,GB-BT38-CARRICKFERGUS BOROUGH COUNCIL,CARRICKFERGUS BOROUGH COUNCIL,BT38,CARRICKFERGUS,DARDNI,Non-productive investments,1580.0,GB,GBP
9755,2014,GB-BT29-DR S PETER FITZGERALD,DR S PETER FITZGERALD,BT29,CRUMLIN,DARDNI,Non-productive investments,3000.0,GB,GBP
9781,2014,GB-BT22-DUNLEATH ESTATES LTD,DUNLEATH ESTATES LTD,BT22,NEWTOWNARDS,DARDNI,Non-productive investments,912.5,GB,GBP
114028,2014,GB-1219336,None,BT62,CRAIGAVON,DARDNI,First afforestation of agricultural land,267.8,GB,GBP


In [23]:
len(df)

1085257

In [24]:
df = df[~df['scheme'].isin([x for x in df['scheme'].value_counts().index if x.endswith('Total')])]
df['scheme'].value_counts()

Single area payment scheme                                                              174828
Agri-environment payments                                                                67393
Payments to farmers in areas with handicaps, other than mountain areas                   25572
First afforestation of agricultural land                                                 10169
Other direct aids                                                                         7663
Non-productive investments                                                                7608
Vocational training and information actions                                               3098
Modernisation of agricultural holdings                                                    2755
Participation of farmers in food quality schemes                                          2746
Infrastructure related to the development and adaptation of agriculture and forestry      2479
Additional amounts of aid                         

In [25]:
df.to_csv('gb_%s.csv' % YEAR, index=False, encoding='utf-8')